In [1]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
import socket
import traceback
import time 
import errno


SERVER_HOST = '127.0.0.1'
SERVER_PORT = 2030

In [2]:
class Client:
    def __init__(self, client_name, client_port):
        self.client_name = client_name
        self.client_ip = None
        self.client_port = client_port
        self.server_PK = None  # The server's public key, to encrypt the messages while sending to the server.
        self.cipher = None 
        
         # Prefix building
        ip = '_'.join(SERVER_HOST.split('.'))
        port = str(SERVER_PORT)
        self.prefix = f'{ip} {port}'

        self.messages_sent = []
        self.messages_received = []

        self.client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # The connection socket
        self.run_client()

    def run_client(self):
        self.client.connect((SERVER_HOST, SERVER_PORT))        
        PK = self.client.recv(1024).decode()  # Get the public key of the server.
        self.server_PK = RSA.import_key(PK)
        self.cipher = PKCS1_OAEP.new(self.server_PK)

        
        print(f"Server's public key is yours, {PK}")
        user_input = ''
        
        while not 'exit' in user_input.lower():
            user_input = input('Message:')
            self.messages_sent.append(user_input)
            message_to_server = self.prefix + ' ' + user_input  # Concatenate the prefix to the actual message
            message_to_server_encoded = message_to_server.encode()
            
            # The encryption part uses the public key of the server
            encrypted_message = self.cipher.encrypt(message_to_server_encoded)
            self.client.send(encrypted_message)

            self.receive_data()
            
        
        self.client.send(user_input.lower().encode()) # Sent the exit command to the server
        self.client.close()
        
    def receive_data(self, timeout=1.5):
        self.client.setblocking(False)
        start_time = time.time()
        while True:
            try:
                data_received = self.client.recv(1024).decode()
                if data_received:
                    print(f"Response: {data_received}")
            except socket.error as e:
                # Handle non-blocking socket exception
                if e.errno == errno.EWOULDBLOCK:
                    pass  # No data available yet
                else:
                    print("Error:", e)
                    traceback.print_exc()
            
            # Check timeout
            if time.time() - start_time >= timeout:
                break
        self.client.setblocking(True)  # Restore blocking mode

In [3]:
c = Client(client_name='alice', client_port=1234)

Server's public key is yours, -----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAyg30OXEwAKRrjUCLFw3p
zgYjMXHm7KEMrsbG8lOvtxIELUuq+bwpNvNbNFGdZ4XlC+aKqVB8rEwO8Y3UgUB1
QEPIwkZvPt7Dd7BPXJH3D9RjUoUAL3Nkd/6wNhEQvav8hxxrzlq49w7PXyKR6nfz
Tl8t4RVHvmEC7TEwlgOK0AzHcVKG84bud6GRN5SDpDGrye7OogFUi5TRbIU91rvT
29ImJ/gQk0jPsTAjZToDmSGi97q8wNNWPqInk8nqe7tEOpI3UhFiNJmdaEp1a/B+
KDP4rFwBITBFpT+ExWvwbjd4TDVdtqrjg5usnhF3pIMYAXe/3QX0XRstkMDxum15
cQIDAQAB
-----END PUBLIC KEY-----


Message: hella
Message: hella
Message: te amo bitch


Response: 127_0_0_1 2030 hella
Response: 127_0_0_1 2030 hella127_0_0_1 2030 te amo bitch


Message: holla 
Message: we dem boyz


Response: 127_0_0_1 2030 holla 
Response: 127_0_0_1 2030 we dem boyz


KeyboardInterrupt: Interrupted by user